In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/transfer_learning.hdf5'
training_epochs = 50
batch_size = 20
images_size = 224

.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  0%|          | 1/1408 [00:00<02:33,  9.18it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:03<00:00, 110.30it/s]


In [4]:
# external_model = InceptionV3(weights='imagenet', include_top=False)
# external_model = ResNet50(weights='imagenet', include_top=False)
external_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
print(".. constructing the transfer model ..")

for layer in external_model.layers:
    layer.trainable = False

temp = Conv2D(filters=128, kernel_size=2, padding='same')(external_model.layers[-13].output)

temp = BatchNormalization()(temp)
temp = Activation('relu')(temp)
temp = MaxPooling2D(pool_size=(2, 2))(temp)
temp = GlobalAveragePooling2D()(temp)
predictions = Dense(2, activation='softmax')(temp)

model = Model(input=external_model.input, output=predictions)

model.summary()    

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    

.. constructing the transfer model ..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56

/Users/pmartin/ml/venv/udacity-capstone/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [6]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/1
1408/1408 [==============================] - 686s 487ms/step - loss: 0.3382 - acc: 0.8295 - val_loss: 0.6924 - val_acc: 0.6307

Epoch 00001: val_loss improved from inf to 0.69241, saving model to ./saved-models/transfer_learning.hdf5


In [7]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [8]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/440 [00:00<?, ?it/s]

.. testing the model on Test data ..


100%|██████████| 440/440 [01:34<00:00,  4.58it/s]


In [9]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/1408 [00:00<?, ?it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [05:04<00:00,  4.69it/s]


In [10]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/352 [00:00<?, ?it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [01:14<00:00,  4.73it/s]


In [11]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 22.4 seconds
training duration: 686.9 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 95.0 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  81
true_negatives:  206
false_positives:  153
false_negatives:  0
RECALL: 1.00
SPECIFICITY: 0.57
ACCURACY: 0.65
PRECISION: 0.35
F1 SCORE: 0.51
FP RATE / ERROR I: 0.43
FN RATE / ERROR II: 0.00
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 305.0 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  314
true_negatives:  578
false_positives:  514
false_negatives:  2
RECALL: 0.99
SPECIFICITY: 0.53
ACCURACY: 0.63
PRECISION: 0.38
F1 SCORE: 0.55
FP RATE / ERROR I: 0.47
FN RATE / ERROR II: 0.01
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:

pr

## False Positives (first 20):

In [12]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [14]:
display_images(false_negative_images[:20])